In [ ]:
import torch
import numpy as np
import os.path as osp
from functools import partial

from changeit3d.in_out.changeit3d_net import prepare_input_data
from changeit3d.in_out.language_contrastive_dataset import LanguageContrastiveDataset
from changeit3d.in_out.pointcloud import pc_loader_from_npz, uniform_subsample
from changeit3d.in_out.basics import pickle_data
from changeit3d.in_out.basics import create_logger
from changeit3d.in_out.arguments import parse_evaluate_changeit3d_arguments

from changeit3d.utils.basics import parallel_apply
from changeit3d.models.model_descriptions import load_pretrained_changeit3d_net
from changeit3d.models.model_descriptions import load_pretrained_pc_ae

from changeit3d.evaluation.auxiliary import pc_ae_transform_point_clouds, sgf_transform_point_clouds
from changeit3d.external_tools.sgf.loader import initialize_and_load_sgf
from changeit3d.utils.visualization import visualize_point_clouds_3d_v2, plot_3d_point_cloud


%load_ext autoreload
%autoreload 2

##
## Manually specify the underlying files using our pretrained nets or yours
##

shape_generator_type = "sgf"

top_data_dir = '../../data'
shape_talk_file = f'{top_data_dir}/shapetalk/language/shapetalk_preprocessed_public_version_0.csv'
vocab_file = f'{top_data_dir}/shapetalk/language/vocabulary.pkl'
top_pc_dir = f'{top_data_dir}/shapetalk/point_clouds/scaled_to_align_rendering'
pretrained_oracle_listener = f'{top_data_dir}/pretrained/listeners/oracle_listener/all_shapetalk_classes/rs_2023/listener_dgcnn_based/ablation1/best_model.pkl'
pretrained_shape_classifier =  f'{top_data_dir}/pretrained/pc_classifiers/rs_2022/all_shapetalk_classes/best_model.pkl'
shape_part_classifiers_top_dir = f'{top_data_dir}/pretrained/part_predictors/shapenet_core_based'
latent_codes_file = f'{top_data_dir}/pretrained/shape_latents/{shape_generator_type}_latent_codes.pkl'


### PC-AE based:    
if shape_generator_type == "pcae":    
    pretrained_shape_generator = f'{top_data_dir}/pretrained/pc_autoencoders/pointnet/rs_2022/points_4096/all_classes/scaled_to_align_rendering/08-07-2022-22-23-42/best_model.pt'
    ## selected PC-AE ablation:         
    selected_ablation = 'decoupling_mag_direction/idpen_0.05_sc_True/' # decoupled and with self-contrast=True
    
### SGF-AE based:        
if shape_generator_type == "sgf":    
    selected_ablation = 'decoupling_mag_direction/idpen_0.05_sc_True/'        
    sub_sample_dataset = '100' # just evaluate over 100 randomly selected test shapes

pretrained_changeit3d = f'{top_data_dir}/pretrained/changers/{shape_generator_type}_based/all_shapetalk_classes/{selected_ablation}/best_model.pt'

In [ ]:
notebook_arguments = []

notebook_arguments.extend(['-shape_talk_file', shape_talk_file])
notebook_arguments.extend(['-latent_codes_file', latent_codes_file])
notebook_arguments.extend(['-vocab_file', vocab_file])
notebook_arguments.extend(['-pretrained_changeit3d', pretrained_changeit3d])
notebook_arguments.extend(['-top_pc_dir', top_pc_dir])
notebook_arguments.extend(['--shape_generator_type', shape_generator_type])
notebook_arguments.extend(['--pretrained_oracle_listener', pretrained_oracle_listener])
notebook_arguments.extend(['--pretrained_shape_classifier', pretrained_shape_classifier])
notebook_arguments.extend(['--shape_part_classifiers_top_dir', shape_part_classifiers_top_dir])

if 'pretrained_shape_generator' in  locals():
    notebook_arguments.extend(['--pretrained_shape_generator', pretrained_shape_generator])

if 'sub_sample_dataset' in  locals():
    notebook_arguments.extend(['--sub_sample_dataset', sub_sample_dataset])    
    

args = parse_evaluate_changeit3d_arguments(notebook_arguments)
logger = create_logger(args.log_dir)